Auteur: Michaël Leroy

Creation du graph routier:
    - recuperation pa dep
    - ajout dstance et durées avant simplification
    - simplification 
    - concatenation de tous les graph departementaux

Sortie:
    geodataframe:
        - nodes : intersections
        - edges : reseau routier    

[+] API: 
    
    - Nominatim: https://nominatim.openstreetmap.org/
        * obtention des surfaces en shapely geometries
    - OpenStreetMaps :
        * Lib OSMnx, obtention des résaux routiers sous forme de graph 
    - OpenChargeMap : 
        * Points de charge EV (data.gouv et autres)        

[+] Méthodologie

    - Récupération des réseaux routiers:
        * Nodes: position des intersections routières
        * Edges: shapely LineString, distance, limitation de vitesse, types de voies...
  (  - Récupération des bornes EV
        * Création d'un noeud sur l'edge le plus proche
        * Split Linestring
        # Branchement du noeud et raccord aus deux noeud 'parents' ) le code est présent mais plus utilisé

[+] Est produit:
        
        * Un graph avec:
            Nodes : intersections routières
            Edges : tronçons entre intersections

[+] Limitations:

        * On ne peut pas arriver à avoir au moins une route par commune, les orphelines sont ratachées à la route la plus proche dans la suite du traitement

[+] Utilité

    - Graph Stats:
        betweeness, closeness.....

[+] Récupération des données par département pour éviter OOM 

https://wiki.openstreetmap.org/wiki/Key:highway

In [ ]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = True

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
        #  '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    

In [ ]:
CFG.area_name

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'
%load_ext autoreload
%autoreload



%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    'Corse-du-Sud': '2A',
    'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
AREA

# Listes des découpages Régions Départements

In [ ]:
# selected_regions = [
#     # 'Larzicourt',
#     # 'Arrigny'
#     # 'Reims'
#     # 'Lot',
#     # 'Aube',
#     'Ardennes',
#     # 'Marne',
#     # 'Haute-Marne',
#     # 'Côtes-d\'Armor',
#     # 'Bretagne',
#     # 'Île-de-France'
#     # 'Grand Est', 
#     # 'Hauts-de-France',
#     # 'Bourgogne-Franche-Comté',
#     # 'Auvergne-Rhône-Alpes',
#     # 'Provence-Alpes-Côte d\'Azur'

# ]
# # selected_regions = CFG.area

In [ ]:
try:
    selection = []
    for reg in CFG.area:
        selection += Administratives[reg]
except:
    print('Not a Région')
    selection= CFG.area
selection

# Graph construction

## Get Graph

In [ ]:
CFG.roads

In [ ]:
# By graphs
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]graph.graphml')
try:
    print(f'Loading graphfor {CFG.area_name} from disk...', end='  ')
    G = ox.io.load_graphml(file)
    print(f'{G.number_of_nodes()} nodes, {G.number_of_edges()} edges')

except:
    print('Datas not found')
    G = nx.MultiDiGraph()
    stats = {}
    total = len(AREA)
    print(CFG.roads)
    for n, area in enumerate(AREA):
        print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
        G_ = get_graph(area, custom_filter=CFG.roads)
        G_ = fill_missing_edge_geometry(G_)
        G = nx.compose(G, G_)
        stats[area] = (G_.number_of_nodes(), G_.number_of_edges()   )
        del G_
    display(stats)
    print(f'before:          {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.')
    G =  ox.utils_graph.get_largest_component(G, strongly=True)
    print(f'keep largest:    {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.')
    G =  ox.utils_graph.remove_isolated_nodes(G)
    print(f'remove isolated: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.')

    ox.io.save_graphml(G, file) 

nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
# G.number_of_nodes(), G.number_of_edges()        


In [ ]:
nodes.shape, edges.shape

In [ ]:
joblib.dump(nodes, os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes.pkl'),compress=3)

joblib.dump(edges, os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges.pkl'),compress=3)


In [ ]:
# graph = ox.graph_from_gdfs(nodes, edges)

In [ ]:
# del G, nodes, edges

In [ ]:
**** fin du traitement des données ****

In [ ]:
def missing(df):
    return pd.DataFrame({'column_name': df.columns,
                         'percent_missing': df.isnull().sum() * 100 / len(df)
                         }).sort_values(by='percent_missing',ascending=True)

In [ ]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'amenity': True}, 
                                     which_result=None, buffer_dist=None)
missing(test)

In [ ]:
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]amenity_tags.feather')
try:
  tags = gpd.read_feather(file)
except:
    print('Datas not found on disk')
    tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      tags = pd.concat(
        [tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'amenity': True}, 
                                        which_result=None, buffer_dist=None)[['amenity','geometry']]
        ]
      )
    tags.to_feather(file)

display(tags.info())
display(tags['amenity'].value_counts())

del tags

In [ ]:
# file = os.path.join(CFG.data_path, f'[{CFG.area_name}]social_tags.feather')
# try:
#   social_tags = gpd.read_feather(file)
# except:
#     print('Datas not found on disk')
#     social_tags = gpd.GeoDataFrame()
#     total = len(AREA)
#     for n, area in enumerate(AREA):
#       print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
#       social_tags = pd.concat(
#         [social_tags,
#          ox.geometries.geometries_from_place(area + ', Metropolitan France',
#                                         tags={'amenity':['food_court',
#                                                           'restaurant',
#                                                           'arts_centre',
#                                                           'cinema',
#                                                           'community_centre',
#                                                           'conference_centre',
#                                                           'events_venue',
#                                                           'exhibition_center',
#                                                           'music_venue',
#                                                           'planetarium',
#                                                           'theatre'],
#                                               }, 
#                                         which_result=None, buffer_dist=None)[['amenity','geometry',]]
#         ]
#       )
#     # social_tags = social_tags[['amenity','geometry',]]
#     social_tags.to_feather(file)

# display(social_tags.info())
# display(social_tags['amenity'].value_counts())

# del social_tags

# Historic

In [ ]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'historic':True }, 
                                     which_result=None, buffer_dist=None)
missing(test)

In [ ]:
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]historic_tags.feather')
try:
  historic_tags = gpd.read_feather(file)
except:
    print('Datas not found on disk')
    historic_tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      historic_tags = pd.concat(
        [historic_tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'historic':True }, 
                                     which_result=None, buffer_dist=None)[['historic','geometry']]
        ]
      )
    # historic_tags = historic_tags[['historic','amenity','geometry','source']]
    historic_tags.to_feather(file)                                   

display(historic_tags.info())
display(historic_tags['historic'].value_counts())

del historic_tags

# Natural

In [ ]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'natural': True}, 
                                     which_result=None, buffer_dist=None)
missing(test)

In [ ]:

file = os.path.join(CFG.data_path, f'[{CFG.area_name}]natural_tags.feather')
try:
  natural_tags = gpd.read_feather(file)
except:
    print('Datas not found on disk')
    natural_tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      natural_tags = pd.concat(
        [natural_tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'natural':True }, 
                                     which_result=None, buffer_dist=None)[['natural','geometry']]
        ]
      )
    natural_tags.to_feather(file)                                   

display(natural_tags.info())
display(natural_tags['natural'].value_counts())

del natural_tags

# Charge Stations

In [ ]:
# test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
#                                         tags={'amenity':['charging_station']}, 
#                                      which_result=None, buffer_dist=None)
# missing(test)

In [ ]:
# file = os.path.join(CFG.data_path, f'[{CFG.area_name}]charge_tags.feather')
# try:
#   charge_tags = gpd.read_feather(file)
# except:
#     print('Datas not found on disk')
#     charge_tags = gpd.GeoDataFrame()
#     total = len(AREA)
#     for n, area in enumerate(AREA):
#       print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
#       charge_tags = pd.concat(
#         [charge_tags,
#          ox.geometries.geometries_from_place(area + ', Metropolitan France',
#                                         tags={'amenity':['charging_station']}, 
#                                      which_result=None, buffer_dist=None)
#                                     #  [[
#                                     # 'amenity', 
#                               #  'capacity',
#                               #  'geometry',
#       #                          'socket:type2',
#       #                          'socket:typee',
#       #                          'socket:type3', 'source','socket:tesla_supercharger',
#       #  'socket:tesla_supercharger:output','socket:tesla_supercharger_ccs:output',
#       #  'socket:type2_combo', 'socket:type2_combo:output','socket:chademo',
#       #  'socket:type3c',
#       # 'amperage',
#       #  'voltage','socket:schuko','source:socket:type2:output', 'source:socket:typee',
#       #  'capacity:charging', 'charging_station:output','socket:chademo:current',
#       #  'socket:type2:current', 'socket:type2_combo:current'
#                       # ]]
#         ]
#       )
#     charge_tags = charge_tags[['amenity', 
#                                'capacity',
#                                'geometry',
#                                'socket:type2',
#                                'socket:typee',
#                                'socket:type3', 'source','socket:tesla_supercharger',
#        'socket:tesla_supercharger:output','socket:tesla_supercharger_ccs:output',
#        'socket:type2_combo', 'socket:type2_combo:output','socket:chademo',
#        'socket:type3c','amperage',
#        'voltage','socket:schuko','source:socket:type2:output', 'source:socket:typee',
#        'capacity:charging', 'charging_station:output','socket:chademo:current',
#        'socket:type2:current', 'socket:type2_combo:current'
#                       ]]
#     charge_tags.to_feather(file)                                                

# display(charge_tags.info())
# display(charge_tags['amenity'].value_counts())

# del charge_tags

In [ ]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'leisure':True}, 
                                     which_result=None, buffer_dist=None)
missing(test)

In [ ]:
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]leisure_tags.feather')
try:
  leisure_tags = gpd.read_feather(file)
except:
    print('Datas not found on disk')
    leisure_tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      leisure_tags = pd.concat(
        [leisure_tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'leisure':True}, 
                                     which_result=None, buffer_dist=None)
                                     [
                                                                          [
                                                                          'leisure',
                                                                          'geometry',
                                                                          ]
                                                                           ]
        ]
      )
    leisure_tags.to_feather(file)                                    

display(leisure_tags.info())
display(leisure_tags['leisure'].value_counts())

del leisure_tags

In [ ]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'sport':True}, 
                                     which_result=None, buffer_dist=None)
missing(test)

In [ ]:
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]sport_tags.feather')
try:
  sport_tags = gpd.read_feather(file)
except:
    print('Datas not found on disk')
    sport_tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      sport_tags = pd.concat(
        [sport_tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'sport':True}, 
                                     which_result=None, buffer_dist=None)
                                     [
                                                                      ['sport', 
                                                                       'geometry']
                                                                        ]
        ]                                                               
      )
    
    sport_tags.to_feather(file)
                                  

display(sport_tags.info())
display(sport_tags['sport'].value_counts())

del sport_tags

In [ ]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'boundary':'administrative'}, 
                                     which_result=None, buffer_dist=None)
missing(test)

In [ ]:
# TODO: rename file with []
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]boundaries.feather')
try:
  boundaries = gpd.read_feather(file)
except:
    print('Datas not found')
    boundaries = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      boundaries = pd.concat(
        [boundaries,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'boundary':'administrative'}, 
                                     which_result=1, buffer_dist=None)
                    #                  [['name', 
                    #            'geometry',
                    #   ]]
        ]
      )
    boundaries = boundaries[[
      'boundary','geometry','admin_level'
    ]]
    boundaries.to_feather(file)                                  

display(boundaries.info())
# tourism_tags['amenity'].value_counts()   
boundaries                                 

In [ ]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'tourism': True}, 
                                     which_result=None, buffer_dist=None)
missing(test)

In [ ]:
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]tourism_tags.feather')
try:
  tourism_tags = gpd.read_feather(file)
except:
    print('Datas not found')
    tourism_tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      tourism_tags = pd.concat(
        [tourism_tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'tourism': True}, 
                                     which_result=None, buffer_dist=None)
                                     [['tourism', 
                               'geometry',
                      ]]
        ]
      )
   
    tourism_tags.to_feather(file)                                  

display(tourism_tags.info())
display(tourism_tags['tourism'].value_counts())

del tourism_tags

In [ ]:
# TODO: get each commune Mairie coordinates

In [ ]:
# TODO: merge decoupage with osm_id and Mairies coordinates

In [ ]:
*    End of OSM datasets file retrieval

In [ ]:
# Add isom_id to communes decoupage, for graph building

import requests, io
data_path = './datas'
file_name = 'decoupage_administratif'
ext = '.json'
url ='https://www.data.gouv.fr/fr/datasets/r/fb3580f6-e875-408d-809a-ad22fc418581'


# import requests
# from requests.adapters import HTTPAdapter
# import urllib3
# from urllib3 import Retry

# session = requests.Session()
# headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
# session.headers.update(headers)
# adapter = HTTPAdapter(max_retries=Retry(total=10, backoff_factor=1))
# session.mount("http://", adapter)
# session.mount("https://", adapter)

# def get_(url):
#     response = session.get(url.strip())
#     return response.json()
# # GET COORDINATES FROM CITY NAME

# def get_place_osmid(county,city, country):
#     url  =  'https://nominatim.openstreetmap.org/search?'
#     # url += f'q={adress}%2C+'
#     url += f'city={city}&'
#     url += f'country={country}&'
#     url += f'county={county}&'


#     url +=  'format=geojson'#&polygon_geojson=1'
#     return get_(url)


# def get_osm_infos(gdf):
#     total = gdf.shape[0]
#     for i, (n, row) in enumerate(gdf.iterrows()):
#         print(f'{i/total:.2%}                   ', end='\r')

#         data = get_place_osmid('', row['libgeo'], 'France')

#         try:
#             gdf.loc[n, 'osm_id'] = data['features'][0]['properties']['osm_id']
#         except:
#             gdf.loc[n, 'osm_id'] = None
#         try:
#             gdf.loc[n, 'place_id'] = data['features'][0]['properties']['place_id']
#         except:
#             gdf.loc[n, 'place_id'] = None
#         try:
#             gdf.loc[n, 'place_rank'] = data['features'][0]['properties']['place_rank']
#         except:
#             gdf.loc[n, 'place_rank'] = None
#         try:
#             gdf.loc[n, 'display_name'] = data['features'][0]['properties']['display_name']
#         except:
#             gdf.loc[n, 'display_name'] = None
#         try:
#             gdf.loc[n, 'importance'] = data['features'][0]['properties']['importance']
#         except:
#             gdf.loc[n, 'importance'] = None
            
#     return gdf


# try: 
#     print('Loading data from local file...')
#     decoupage = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
    
# except:
#     print('Loading data from url...')

#     s = requests.get(url).content
#     decoupage = pd.read_csv(io.StringIO(s.decode('utf-8')))
    
#     print('Saving data to local file...')
#     decoupage.to_feather(os.path.join(data_path,file_name + '.feather'))

# # Check if osm_id is present and complete gdf if needed
# if 'osm_id' not in decoupage.columns:
#     print('Adding osm feature (approx 5 hours to complete!!!)')
#     decoupage = get_osm_infos(decoupage)
#     print('Saving data to local file...')
#     decoupage.to_feather(os.path.join(data_path,file_name + '.feather'))

# display(decoupage.info())
# display(decoupage.head())

In [ ]:
data_path = './datas'
file_name = 'decoupage_administratif'
decoupage = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))

In [ ]:
decoupage.plot()

In [ ]:
# import requests
# from requests.adapters import HTTPAdapter
# import urllib3
# from urllib3 import Retry

# session = requests.Session()
# headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
# session.headers.update(headers)
# adapter = HTTPAdapter(max_retries=Retry(total=10, backoff_factor=1))
# session.mount("http://", adapter)
# session.mount("https://", adapter)

# def get_(url):
#     response = session.get(url.strip())
#     return response.json()
# # GET COORDINATES FROM CITY NAME

# def get_place_osmid(county,city, country):
#     url  =  'https://nominatim.openstreetmap.org/search?'
#     # url += f'q={adress}%2C+'
#     url += f'city={city}&'
#     url += f'country={country}&'
#     url += f'county={county}&'


#     url +=  'format=geojson'#&polygon_geojson=1'
#     return get_(url)


In [ ]:
# import concurrent.futures



# def get_osm_infos(gdf):
#     total = gdf.shape[0]
#     for i, (n, row) in enumerate(gdf.iterrows()):
#         print(f'{i/total:.2%}                   ', end='\r')
#         data = get_place_osmid('', row['libgeo'], 'France')
#         gdf.loc[n, 'osm_id'] = data['features'][0]['properties']['osm_id']
#         gdf.loc[n, 'place_id'] = data['features'][0]['properties']['place_id']
#         gdf.loc[n, 'place_rank'] = data['features'][0]['properties']['place_rank']
#         gdf.loc[n, 'display_name'] = data['features'][0]['properties']['display_name']
#         gdf.loc[n, 'importance'] = data['features'][0]['properties']['importance']
#     return gdf


# def multi_process_osmid(gdf, processes=4):
#     # Split the DataFrame into four equal chunks
#     num_chunks = processes
#     chunk_size = len(gdf) // num_chunks
#     gdf_chunks = [gdf[i:i+chunk_size].copy() for i in range(0, len(gdf), chunk_size)]

#     # Create a ThreadPoolExecutor to process chunks asynchronously
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         # Submit the processing function for each chunk
#         futures = [executor.submit(get_osm_infos, chunk) for chunk in gdf_chunks]

#         # Wait for all futures to complete
#         concurrent.futures.wait(futures)

#         # Get the results from the completed futures
#         results = pd.concat([future.result() for future in futures] )
#     return results       


In [ ]:
# decoupage = get_osm_infos(decoupage)

In [ ]:
# decoupage = multi_process_osmid(decoupage, processes=2)